In [99]:
import pandas as pd
import numpy as np

from datetime import datetime
import time

from collections import defaultdict

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


%matplotlib inline

from math import log
from collections import Counter
import re



import os, sys, email
import numpy as np 
import pandas as pd
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')


# Network analysis
import networkx as nx
# NLP
from nltk.tokenize.regexp import RegexpTokenizer

from subprocess import check_output



In [100]:
emails_df = pd.read_csv('emails.csv')
emails_df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [101]:
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [102]:
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
emails_df['content'] = list(map(get_text_from_email, messages))
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages
emails_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [103]:
dict_month = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

def chsnge_date(text):
    date = re.findall(r'\d{1,2}\s\w+\s\d{4}\s\d{2}:\d{2}:\d{2}', text)[0]
    month = re.findall(r'\s(\D+)\s', date)[0]
    date = date.replace(month, dict_month[month])
    date_list = date.split(' ')
    day = data_list[0]
    month = data_list[1]
    year = 2000+int(date_list[2][-1])
    
    date_ = '.'.join(date_list[:3])
    time = date_list[-1]
    date = ' '.join([date_, time])
    
    

In [104]:
emails_df = emails_df.set_index('Message-ID')\
    .drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], axis=1)

emails_df['Date'] = pd.to_datetime(emails_df['Date'], infer_datetime_format=True)
emails_df.dtypes

Date          datetime64[ns]
From                  object
To                    object
Subject               object
X-From                object
X-To                  object
X-cc                  object
X-bcc                 object
X-Folder              object
X-Origin              object
X-FileName            object
content               object
user                  object
dtype: object

In [105]:
emails_df

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
Message-ID,,,,,,,,,,,,,
<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,(phillip.allen@enron.com),(tim.belden@enron.com),,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,(phillip.allen@enron.com),(randall.gay@enron.com),,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p
<30965995.1075863688265.JavaMail.evans@thyme>,2000-08-31 11:17:00,(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Greg,\n\n How about either next Tuesday or Thu...",allen-p
<16254169.1075863688286.JavaMail.evans@thyme>,2000-08-22 14:44:00,(phillip.allen@enron.com),"(david.l.johnson@enron.com, john.shafer@enron....",,Phillip K Allen,"david.l.johnson@enron.com, John Shafer",,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Please cc the following distribution list with...,allen-p
<17189699.1075863688308.JavaMail.evans@thyme>,2000-07-14 13:59:00,(phillip.allen@enron.com),(joyce.teixeira@enron.com),Re: PRC review - phone calls,Phillip K Allen,Joyce Teixeira,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,any morning between 10 and 11:30,allen-p
<20641191.1075855687472.JavaMail.evans@thyme>,2000-10-17 09:26:00,(phillip.allen@enron.com),(mark.scott@enron.com),Re: High Speed Internet Access,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,1. login: pallen pw: ke9davis\n\n I don't thi...,allen-p


In [106]:
emails_df['TimeAll'] = emails_df['Date'].apply(lambda d: time.mktime(d.timetuple()))
emails_df = emails_df.sort_values(by = 'TimeAll')
emails_df = emails_df.fillna('unknown')

In [107]:
def change_to_from(st):
    if st == 'unknown':
        return 'unknown'
    if len(list(st)) == 1:
        return list(st)
    else:
        return list(st)

In [108]:
emails_df['From'] = emails_df['From'].apply(lambda st: change_to_from(st))
emails_df['To'] = emails_df['To'].apply(lambda st: change_to_from(st))

In [115]:
def get_folder(folders):
    return folders.split('\\')[-1]

In [124]:
f = '\Greg_Whalley_Oct2001\Notes Folders\Notes inbox'.split('\')
e.findall('\.{,100}$', f)

SyntaxError: EOL while scanning string literal (<ipython-input-124-bf7a3f66ee37>, line 1)

In [109]:
emails_df

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,TimeAll
Message-ID,,,,,,,,,,,,,,
<26478103.1075845469034.JavaMail.evans@thyme>,1980-01-01 00:00:00,[outlook.team@enron.com],unknown,3 - URGENT - TO PREVENT LOSS OF INFORMATION,Outlook Migration Team,,,,\Greg_Whalley_Oct2001\Notes Folders\Notes inbox,WHALLEY-G,gwhalley.nsf,Critical Migration Information:\n\n1. Your sch...,whalley-g,3.155220e+08
<6457299.1075840201254.JavaMail.evans@thyme>,1980-01-01 00:00:00,[rosalee.fleming@enron.com],"[barbara.schmolt@enron.com, tracey.florio@enro...",University of Houston Reception,Rosalee Fleming,"Rodolfo AceveDo, Lisa Alderman, Teri Alexander...",,,\Kenneth_Lay_Dec2000\Notes Folders\All documents,LAY-K,klay.nsf,Calling All Alumni!\n \nUniversity of Ho...,lay-k,3.155220e+08
<3823157.1075840203657.JavaMail.evans@thyme>,1980-01-01 00:00:00,[tori.wells@enron.com],[maxwells@train.missouri.org],Re: Update from cousins in Missouri,Tori L Wells,"""Janice"" <maxwells@train.missouri.org>",,,\Kenneth_Lay_Dec2000\Notes Folders\All documents,LAY-K,klay.nsf,Letter dictated by Ken Lay\n\n\n\nHello Janice...,lay-k,3.155220e+08
<3298981.1075840289876.JavaMail.evans@thyme>,1980-01-01 00:00:00,[rosalee.fleming@enron.com],[jsikes@cfr.org],Re: Energy TF Final Report,Rosalee Fleming,JSikes@cfr.org,,,\Kenneth_Lay_June2001\Notes Folders\All documents,LAY-K,klay.nsf,Mr. Sikes -\n\nKenneth L. Lay is the Chairman ...,lay-k,3.155220e+08
<31290580.1075845660660.JavaMail.evans@thyme>,1980-01-01 00:00:00,[kay.mann@enron.com],[brian.barto@enron.com],"Re: BlueDog Change Order #2, Rev 4",Kay Mann,Brian D Barto,,,\Kay_Mann_June2001_1\Notes Folders\All documents,MANN-K,kmann.nsf,"Hi Brian,\n\nHere are my observations/question...",mann-k,3.155220e+08
<8626751.1075849657596.JavaMail.evans@thyme>,1980-01-01 00:00:00,[john.griffith@enron.com],unknown,Don't forget to register!,John Griffith,,,,\John_Griffith_Nov2001\Notes Folders\Discussio...,GRIFFITH-J,jgriffit.nsf,To register your Palm Computingc organizer ele...,griffith-j,3.155220e+08
<2651314.1075846786139.JavaMail.evans@thyme>,1980-01-01 00:00:00,[susan.scott@enron.com],[marcelo.meira@enron.com],Re: ERMS traders in EnPower,Susan M Scott,Marcelo Meira,,,\Susan_Scott_Dec2000_June2001_2\Notes Folders\...,SCOTT-S,sscott5.nsf,\n\tEnron Capital & Trade Resources Corp.\n...,scott-s,3.155220e+08
<26090663.1075858208217.JavaMail.evans@thyme>,1980-01-01 00:00:00,[patrice.mims@enron.com],[hunter.shively@enron.com],Gas Daily Chicago MidPoint Posting for Decembe...,Patrice L Mims,Hunter S Shively,,,\Patrice_Mims-Thurston_Jun2001\Notes Folders\D...,Mims-Thurston-P,pmims.nsf,"For December 21, 2000 activity, I reported a ...",mims-thurston-p,3.155220e+08
<21679093.1075853530267.JavaMail.evans@thyme>,1980-01-01 00:00:00,[kay.mann@enron.com],[communityrelations@enron.com],Volunteer for flood relief,Kay Mann,communityrelations@enron.com,,,\Kay_Mann_Nov2001\Notes Folders\Discussion thr...,Mann-K,kmann.nsf,Kay Mann\nx57566,mann-k,3.155220e+08


In [110]:
emails_df['Year'] = emails_df.Date.apply(lambda x: x.year)
emails_df_2000 = emails_df[emails_df['Year'] == 2000]
emails_df_2000

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,TimeAll,Year
Message-ID,,,,,,,,,,,,,,,
<1525076.1075855927506.JavaMail.evans@thyme>,2000-01-01 14:36:00,[sally.beck@enron.com],[fernley.dyson@enron.com],Happy New Year - No Y2K Fear!,Sally Beck,Fernley Dyson,,,\Sally_Beck_Dec2000\Notes Folders\'sent mail,Beck-S,sbeck.nsf,We are wrapping up several hours in the office...,beck-s,946726560.0,2000
<31981526.1075855819973.JavaMail.evans@thyme>,2000-01-01 14:36:00,[sally.beck@enron.com],[fernley.dyson@enron.com],Happy New Year - No Y2K Fear!,Sally Beck,Fernley Dyson,,,\Sally_Beck_Dec2000\Notes Folders\Discussion t...,Beck-S,sbeck.nsf,We are wrapping up several hours in the office...,beck-s,946726560.0,2000
<14963914.1075855911913.JavaMail.evans@thyme>,2000-01-01 14:36:00,[sally.beck@enron.com],[fernley.dyson@enron.com],Happy New Year - No Y2K Fear!,Sally Beck,Fernley Dyson,,,\Sally_Beck_Dec2000\Notes Folders\Sent,Beck-S,sbeck.nsf,We are wrapping up several hours in the office...,beck-s,946726560.0,2000
<3071171.1075855813238.JavaMail.evans@thyme>,2000-01-01 14:36:00,[sally.beck@enron.com],[fernley.dyson@enron.com],Happy New Year - No Y2K Fear!,Sally Beck,Fernley Dyson,,,\Sally_Beck_Dec2000\Notes Folders\All documents,Beck-S,sbeck.nsf,We are wrapping up several hours in the office...,beck-s,946726560.0,2000
<7129277.1075856793211.JavaMail.evans@thyme>,2000-01-01 19:17:00,[lenos@ucy.ac.cy],[gordon.sick@rogroup.com],Program attached; March NY RO Conference/Parti...,Lenos Trigeorgis <lenos@ucy.ac.cy>,gordon.sick@rogroup.com,gordon.sick@rogroup.com,,\Vincent_Kaminski_Jun2001_7\Notes Folders\Disc...,Kaminski-V,vkamins.nsf,The current version of the conference program ...,kaminski-v,946743420.0,2000
<31317197.1075857040781.JavaMail.evans@thyme>,2000-01-01 19:17:00,[lenos@ucy.ac.cy],[gordon.sick@rogroup.com],Program attached; March NY RO Conference/Parti...,Lenos Trigeorgis <lenos@ucy.ac.cy>,gordon.sick@rogroup.com,gordon.sick@rogroup.com,,\Vincent_Kaminski_Jun2001_9\Notes Folders\Conf...,Kaminski-V,vkamins.nsf,The current version of the conference program ...,kaminski-v,946743420.0,2000
<28718451.1075856785731.JavaMail.evans@thyme>,2000-01-01 19:17:00,[lenos@ucy.ac.cy],[gordon.sick@rogroup.com],Program attached; March NY RO Conference/Parti...,Lenos Trigeorgis <lenos@ucy.ac.cy>,gordon.sick@rogroup.com,gordon.sick@rogroup.com,,\Vincent_Kaminski_Jun2001_6\Notes Folders\All ...,Kaminski-V,vkamins.nsf,The current version of the conference program ...,kaminski-v,946743420.0,2000
<13299574.1075846145808.JavaMail.evans@thyme>,2000-01-02 13:12:00,[andrew.parsons@enron.com],"[philippe.bibi@enron.com, mark.palmer@enron.co...",Summary of Y2K Glitches,Andrew Parsons,"Joseph W Sutton, Richard Causey, Philippe A Bi...","Alberto Gude, Robert Johansen, Karen Denne",,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,Following please find a summary of the minor Y...,kean-s,946807920.0,2000
<18024119.1075846264226.JavaMail.evans@thyme>,2000-01-02 13:12:00,[andrew.parsons@enron.com],"[philippe.bibi@enron.com, mark.palmer@enron.co...",Summary of Y2K Glitches,Andrew Parsons,"Joseph W Sutton, Richard Causey, Philippe A Bi...","Alberto Gude, Robert Johansen, Karen Denne",,\Steven_Kean_Dec2000_1\Notes Folders\Archiving...,KEAN-S,skean.nsf,Following please find a summary of the minor Y...,kean-s,946807920.0,2000


In [111]:
import csv
emails_df.to_csv('Enron_data_all.csv')

In [112]:
emails_df_2000.to_csv('Enron_data_2000.csv')